In [14]:
email_conversation = """From: 박만혜 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [30]:
from pydantic import BaseModel, Field

# 이메일 본문으로부터 주요 엔티티 추출
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    phone_number: str = Field(description="메일 본문에 언급된 전화번호")
    company: str= Field(description="메일을 보낸 사람의 회사명")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [31]:
# LCEL 구조

# chain = prompt | llm | output_parser

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [33]:
from langchain_core.output_parsers import PydanticOutputParser

# PydanticOutputParser 생성
output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [34]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

# QUESTION:
다음의 이메일 내용 중에서 주요 내용을 추출해 주세요.

#EMAIL CONVERSATION:
{email_conversation}

#FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=output_parser.get_format_instructions())

In [35]:
prompt

PromptTemplate(input_variables=['email_conversation'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "phone_number": {"description": "메일 본문에 언급된 전화번호", "title": "Phone Number", "type": "string"}, "company": {"description": "메일을 보낸 사람의 회사명", "title": "Company", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": 

In [36]:
# 체인 생성
chain = prompt | llm | output_parser

In [38]:
answer = chain.invoke({"email_conversation":email_conversation})

In [41]:
query = f"{answer.person} {answer.company} {answer.email}"
query

'김철수 바이크코퍼레이션 chulsoo.kim@bikecorporation.me'

# Serp API - Google 검색

In [24]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
from langchain_community.utilities import SerpAPIWrapper

params = {"engine":"google","gl":"kr","hl":"ko","num":3}

search = SerpAPIWrapper(params=params)

In [43]:
search_result = search.run(query)

In [ ]:
search_result = eval(search_result)

In [47]:
# 검색 결과
search_result_string = '\n'.join(search_result)

In [48]:
answer

EmailSummary(person='김철수', phone_number='', company='바이크코퍼레이션', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리님에게 'ZENESIS' 자전거의 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.", date='1월 15일 오전 10시')

In [58]:
from langchain_core.prompts import PromptTemplate

report_prompt = PromptTemplate.from_template(
    """
    당신은 이메일의 주요 정보를 바탕으로 요약 정리해주는 전문가입니다.
    당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
    주어진 정보를 기반으로 양식(format)에 맞춰서 요약을 작성해주세요.
    답변에는 카테고리별로 emoji를 적극 활용해 답변해주세요.
    
    #information
    - Name : {sender}
    - Additional Information about sender : {additional_information}
    - Company : {company}
    - Email : {email}
    - Subject : {subject}
    - Summary : {summary}
    
    #Format(in markdown format):
    보낸 사람 :
    - (보낸 사람의 이름, 회사정보)
    
    이메일 주소 :
    - (보낸 사람의 이메일 주소)
    
    보낸 사람과 관련하여 검색된 추가 정보:
    - (검색된 추가 정보)
    
    주요 내용:
    - (이메일 제목, 요약)
    
    일정:
    - (미팅 날짜 및 시간)
    
    #Answer:
    """
)

In [59]:
from langchain_core.output_parsers import StrOutputParser

report_chain = report_prompt | ChatOpenAI(model="gpt-4o-mini",temperature=0) | StrOutputParser()

In [60]:
report_response = report_chain.invoke({
    "sender":answer.person,
    "additional_information":search_result_string,
    "company":answer.company,
    "email":answer.email,
    "subject":answer.subject,
    "summary":answer.summary,
    "date":answer.date
})

In [61]:
print(report_response)

```markdown
보낸 사람 :
- 김철수, 상무이사, 바이크코퍼레이션

이메일 주소 :
- chulsoo.kim@bikecorporation.me

보낸 사람과 관련하여 검색된 추가 정보:
- 김철수 상무는 바이크코퍼레이션에서 자전거 유통 및 협력 관련 업무를 담당하고 있습니다. 

주요 내용:
- 제목: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안 🚴‍♂️
- 요약: 김철수 상무가 이은채 대리님에게 'ZENESIS' 자전거의 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다. 📄🤝

일정:
- 미팅 날짜 및 시간: 추후 협의 예정 ⏰
```
